## WORD TO JSON

In [6]:
from docx import Document
import json
import os

input_folder = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data"
output_folder = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".docx"):
        doc_path = os.path.join(input_folder, filename)
        doc = Document(doc_path)
        result = []
        actual = {}
        for para in doc.paragraphs:
            estilo = para.style.name.lower()
            texto = para.text.strip()
            if not texto:
                continue
            if estilo in ["heading 1", "encabezado 1"]:
                if actual:
                    # Limpia el campo "text" antes de guardar el bloque
                    if "text" in actual:
                        actual["text"] = actual["text"].lstrip()
                    result.append(actual)
                    actual = {}
                actual["title"] = texto
                actual["text"] = ""
            elif estilo == "normal":
                if "text" not in actual:
                    actual["text"] = texto
                else:
                    actual["text"] += "\n" + texto
        if actual:
            if "text" in actual:
                actual["text"] = actual["text"].lstrip()
            result.append(actual)

        # Extrae ley_nombre como antes
        ley_nombre = ""
        for bloque in result:
            if "title" in bloque:
                ley_nombre = bloque["title"]
                break
            elif "text" in bloque and not ley_nombre:
                ley_nombre = bloque["text"]

        # Si hay ley_nombre, lo agregas como bloque al principio
        if ley_nombre:
            result.insert(0, {"text": ley_nombre})

        # Construye el output_data SIN ley_nombre como campo aparte
        output_data = {
            "filename": os.path.splitext(filename)[0],
            "bloques": result
        }

        output_path = os.path.join(output_folder, filename.replace(".docx", ".json"))
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)

        print(f"¡Archivo guardado en {output_path}!")

¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Inclusión financiera.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Ley de Residuos.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Ley Defensa al Consumidor.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Licencias Especiales.json!


## WEB SCRAPPING

In [ ]:
import os
import re
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# === CONFIGURACIÓN ===
output_folder = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/web_scrapping_json"
os.makedirs(output_folder, exist_ok=True)

# Lista de URLs a procesar
urls = [
    "https://www.impo.com.uy/inclusionfinanciera/",
    "https://www.impo.com.uy/gestion-de-residuos/",
    "https://www.impo.com.uy/derechosconsumidor/",
    "https://www.impo.com.uy/licenciasespeciales/"
]

def generar_filename(url):
    path = urlparse(url).path.strip("/")
    return path.split("/")[-1].replace("-", "_") or "index"

def extraer_bloques_con_regex(html):
    soup = BeautifulSoup(html, "html.parser")
    divs = soup.find_all("div", class_="wpb_text_column wpb_content_element")
    if len(divs) < 2:
        return []

    div = divs[1]

    # Remover encabezado estándar
    encabezado = [
        "La Ley en tu lenguaje",
        "Programa Lenguaje Ciudadano",
        "Una forma fácil y práctica de comprender las leyes.",
        "(Actualizado: enero, 2024)"
    ]
    for p in div.find_all("p", limit=4):
        text = p.get_text(strip=True)
        if text in encabezado:
            p.decompose()

    # Unificar texto y listas
    raw_text = ""
    for elem in div.descendants:
        if elem.name == "li":
            raw_text += f"- {elem.get_text(strip=True)}\n"
        elif elem.name in ["p", "div", "span"] and elem.get_text(strip=True):
            raw_text += elem.get_text(strip=True) + "\n"

    # Expresión regular para separar preguntas y respuestas
    pattern = re.compile(r"(¿.*?\?)", re.DOTALL)
    parts = pattern.split(raw_text)

    bloques = []
    for i in range(1, len(parts) - 1, 2):
        question = parts[i].strip()
        answer = parts[i + 1].strip()
        bloques.append({
            "question": question,
            "answer": answer
        })

    return bloques

# === PROCESAR TODAS LAS URLS ===
for url in urls:
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error al acceder a {url}")
            continue

        bloques = extraer_bloques_con_regex(response.text)
        if not bloques:
            print(f"No se extrajeron bloques desde: {url}")
            continue

        output_data = {
            "filename": generar_filename(url).replace("_", " ").capitalize(),
            "link": url,
            "bloques": bloques
        }

        output_file = os.path.join(output_folder, generar_filename(url) + ".json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)

        print(f"JSON guardado: {output_file}")
    except Exception as e:
        print(f"Error procesando {url}: {e}")


✅ JSON guardado: C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/web_scrapping_json\inclusionfinanciera.json
✅ JSON guardado: C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/web_scrapping_json\gestion_de_residuos.json
✅ JSON guardado: C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/web_scrapping_json\derechosconsumidor.json
✅ JSON guardado: C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/web_scrapping_json\licenciasespeciales.json
